Is this lmd? or not check this may be useless

In [1]:
import os
import torch
import numpy as np
from tqdm import tqdm 
from datasets import load_dataset

import miditok
# from transformers import AutoModelForCausalLM, MistralConfig, Trainer, TrainingArguments, GenerationConfig, BitsAndBytesConfig
tokenizer = miditok.REMI.from_pretrained("theglassofwater/remi_12500")

dataset = load_dataset("theglassofwater/pretraining_tokenized_dataset_1", split="train") # keep testing data out,
dataset

dataset.remove_columns(["Unnamed: 0"])

/user/HS400/ye00098/.local/lib/python3.10/site-packages/miditok/midi_tokenizer.py:3252: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  self.config = TokenizerConfig()
/user/HS400/ye00098/.local/lib/python3.10/site-packages/miditok/classes.py:702: UserWarning: The special token PAD_None is present twice in your configuration. Skipping its duplicated occurrence.
  return cls(**input_dict, **kwargs)


OSError: [Errno 28] No space left on device

In [12]:
## seeing whats in the data 
training = dataset["train"]
sample = training["text"][10]
sample[:240]

'PIECE_START STYLE=JSFAKES GENRE=JSFAKES TRACK_START INST=48 BAR_START NOTE_ON=59 TIME_DELTA=4 NOTE_OFF=59 NOTE_ON=64 TIME_DELTA=2 NOTE_OFF=64 NOTE_ON=63 TIME_DELTA=2 NOTE_OFF=63 NOTE_ON=64 TIME_DELTA=8 NOTE_OFF=64 BAR_END BAR_START NOTE_ON='

In [18]:
### Tokenization process 
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer


In [26]:
tokenizer = Tokenizer(BPE(),)
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.pre_tokenizer.pre_tokenize_str(sample[:100])

[('PIECE_START', (0, 11)),
 ('STYLE', (12, 17)),
 ('=', (17, 18)),
 ('JSFAKES', (18, 25)),
 ('GENRE', (26, 31)),
 ('=', (31, 32)),
 ('JSFAKES', (32, 39)),
 ('TRACK_START', (40, 51)),
 ('INST', (52, 56)),
 ('=', (56, 57)),
 ('48', (57, 59)),
 ('BAR_START', (60, 69)),
 ('NOTE_ON', (70, 77)),
 ('=', (77, 78)),
 ('59', (78, 80)),
 ('TIME_DELTA', (81, 91)),
 ('=', (91, 92)),
 ('4', (92, 93)),
 ('NOTE_O', (94, 100))]

In [28]:
tokenizer.train(files=[] , trainer=trainer)
## making iterater 
def iterate_dataset(dataset, step=512):
    for i in range(0,len(training), step):
        yield training["text"][i:i+step]

len(training)
tokenizer.train_from_iterator(iterator=iterate_dataset(training), trainer= trainer)

In [34]:
encoded = tokenizer.encode(sample[:100])
print(encoded.tokens, encoded.ids)
# tokenizer.pre_tokenizer.pre_tokenize_str(sample[:100])

['PIECE_START', 'STYLE', '=', 'JSFAKES', 'GENRE', '=', 'JSFAKES', 'TRACK_START', 'INST', '=', '48', 'BAR_START', 'NOTE_ON', '=', '59', 'TIME_DELTA', '=', '4', 'NOTE_O'] [125, 123, 15, 110, 126, 15, 110, 88, 89, 15, 91, 61, 50, 15, 79, 48, 15, 9, 40]


In [55]:
def gen_vocab(dataset): # making own tokenizer but ignore for now 
    vocab = set()
    for chunk in dataset:
        for row in tqdm(dataset[chunk]):
            listy = row["text"].split(" ")
            vocab = list(vocab)+listy
            vocab = set(vocab)
    vocab = list(vocab)
    vocab.sort()
    vocab = ["[UNK]", "[PAD]", "[SOS]", "[EOS]", "[TDR]"] + vocab
    return vocab
vocab = gen_vocab(dataset)

100%|██████████| 603/603 [00:00<00:00, 12494.09it/s]


In [36]:
print(vocab[:10])

['BAR_END', 'BAR_START', 'GENRE=JSFAKES', 'INST=0', 'INST=24', 'INST=32', 'INST=48', 'NOTE_OFF=36', 'NOTE_OFF=37', 'NOTE_OFF=38']


In [39]:
def encode_token(token, vocab):
    token = vocab.index(token) if token in vocab else "[UNK]"
    return token
def tokenize(text, vocab):
    text = text.split(" ")
    output = list()
    for token in text:
        output.append(encode_token(token,vocab))
    return output

In [56]:
# print(sample.split(" ")[:10])
encoded_song = tokenize(sample, vocab)

264


119

In [4]:
from transformers import AutoTokenizer

og_tokenizer = AutoTokenizer.from_pretrained("TristanBehrens/js-fakes-4bars")
og_vocab = og_tokenizer.get_vocab()
len(og_vocab)

119

In [5]:
context_length = 256
def tokenize(element):
  outputs = og_tokenizer(
      element["text"],
      truncation=True,
      max_length=context_length,
      padding=False
  )
  return {"input_ids": outputs["input_ids"]}

tokenized_datasets = dataset.map(
    tokenize, batched=True, remove_columns=dataset["train"].column_names
)
tokenized_datasets["train"]["input_ids"][0][:10]
tokenized_datasets["train"][0]["input_ids"][:10]

Map:   0%|          | 0/3413 [00:00<?, ? examples/s]

Map: 100%|██████████| 603/603 [00:00<00:00, 3425.46 examples/s]


[77, 78, 75, 9, 76, 8, 37, 5, 36, 92]

In [6]:
from transformers import AutoConfig, GPT2LMHeadModel

In [7]:
context_length = 256
n_layer = 6
n_head = 8
n_emb = 128 # 512

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(og_tokenizer),
    n_positions=context_length,
    n_layer=n_layer,
    n_head=n_head,
    pad_token_id=og_tokenizer.pad_token_id,
    bos_token_id=og_tokenizer.bos_token_id,
    eos_token_id=og_tokenizer.eos_token_id,
    n_embd=n_emb
)



In [8]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")
model

GPT-2 size: 1.2M parameters


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(119, 128)
    (wpe): Embedding(256, 128)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=128, out_features=119, bias=False)
)

In [9]:
from transformers import DataCollatorForLanguageModeling, DataCollatorForSeq2Seq

data_collator = DataCollatorForLanguageModeling(og_tokenizer, mlm=False)


In [168]:
import torch
from torch.utils.data import DataLoader, Dataset


# class CustomDataset(Dataset):
#     def __init__(self, inputs):
#         self.inputs = inputs
    
#     def __len__(self):
#         return len(self.inputs)
    
#     def __getitem__(self, idx):
#         return torch.tensor(self.inputs[idx], dtype=torch.long)


# train_dataset = CustomDataset(tokenized_datasets["train"]["input_ids"])

# # Define data loader
# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Define optimizer



data_loader = DataLoader(dataset=tokenized_datasets["train"], batch_size=256 , collate_fn=data_collator)
# data_loader = DataLoader(dataset=tokenized_datasets["train"], collate_fn=data_collator)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

device = "cpu"

print(len(data_loader))
my_data_loader = torch.tensor([])
for batch in data_loader:
    print(batch)
    x = batch
    break

14
{'input_ids': tensor([[77, 78, 75,  ...,  3,  3,  3],
        [77, 78, 75,  ...,  3,  3,  3],
        [77, 78, 75,  ...,  5, 18, 33],
        ...,
        [77, 78, 75,  ...,  3,  3,  3],
        [77, 78, 75,  ...,  3,  3,  3],
        [77, 78, 75,  ...,  3,  3,  3]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[  77,   78,   75,  ..., -100, -100, -100],
        [  77,   78,   75,  ..., -100, -100, -100],
        [  77,   78,   75,  ...,    5,   18,   33],
        ...,
        [  77,   78,   75,  ..., -100, -100, -100],
        [  77,   78,   75,  ..., -100, -100, -100],
        [  77,   78,   75,  ..., -100, -100, -100]])}


In [28]:
len(x["input_ids"][0])

249

In [95]:
# Training loop
epochs = 1
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        inputs = batch.to(device)  
        target = batch["labels"].to(device)
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    average_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch+1}, Average Loss: {average_loss}")

In [165]:
model.eval()

with torch.no_grad():
    for batch in data_loader:
        inputs = batch.to(device)
        target = batch["labels"].to(device)
        outputs = model(**inputs)
        break
print(outputs)
# outputs["logits"]

CausalLMOutputWithCrossAttentions(loss=tensor(4.8139), logits=tensor([[[ 2.9013e-01, -3.6709e-01,  3.2135e-01,  ..., -4.0565e-01,
           2.2524e-01, -1.4024e-01],
         [-4.3535e-02, -1.1704e-01,  2.4102e-01,  ..., -3.5692e-01,
           4.4924e-01,  4.2927e-04],
         [ 2.9909e-01,  1.3378e-02,  3.0407e-01,  ...,  3.1635e-01,
           1.1818e-01, -8.6370e-02],
         ...,
         [ 2.6882e-01, -4.3566e-01,  3.7479e-01,  ..., -3.9720e-02,
           1.6859e-01, -2.3262e-01],
         [ 2.2972e-01, -3.0511e-02,  5.5130e-02,  ..., -1.2965e-02,
           2.9133e-01,  1.9498e-02],
         [ 1.6271e-01,  7.4397e-02,  2.1400e-01,  ..., -7.4111e-01,
           4.1034e-02, -2.3396e-01]],

        [[ 2.9013e-01, -3.6709e-01,  3.2135e-01,  ..., -4.0565e-01,
           2.2524e-01, -1.4024e-01],
         [-4.3535e-02, -1.1704e-01,  2.4102e-01,  ..., -3.5692e-01,
           4.4924e-01,  4.2927e-04],
         [ 2.9909e-01,  1.3378e-02,  3.0407e-01,  ...,  3.1635e-01,
           1.1

In [166]:
output_tokens = torch.tensor([[0]*256]*256)
output_tokens.shape

torch.Size([256, 256])

In [176]:
# print(outputs["logits"][0], len(outputs["logits"][0]))
print(outputs["logits"].shape)
print(x["input_ids"].shape)

for i in range(len(outputs["logits"])):
    output_tokens[i] = torch.argmax(outputs["logits"][i], dim=-1)

print(output_tokens[0])



torch.Size([256, 256, 119])
torch.Size([256, 256])
tensor([ 77,  78,  75,   9,  76,   8,  37,   5,  36,  92,   5,  91,  63,   5,
         62,  48,   6,  49,  63,   6,  62,   7,   8,  68,   5,  18,  48,  19,
         49,  37,   5,  36,   7,   8,  37,   5,  36,  13,   5,  14,  12,   5,
         11,  27,   5,  26,   7,   8,  37,   6,  36,  27,   6,  26,  12,  19,
         59,  13,   5,  14,   7,  10,   9,  79,   8,  33,   5,  32,  41,   5,
         40,  33,   5,  32,  45,   5,  44,   7,   8,  56,   5,  57,  45,  19,
         44,  95,   5,  70,   7,   8,  33,  14,  32,  41,   5,  40,  34,   5,
         35,  45, 107,  44,  56,   6,  57,   7,   8,  55,   6,  54,  58,   6,
         59,  71,   5,  70,  58,   5,  59,  96,   5,  95,   7,  10,   9,  74,
          8,  13,   5,  14,  13,   5,  14,  13,   5,  14,  13,   5,  14,   7,
          8,  12,   5,  11,  13,  19,  14,  13,   5,  14,   7,   8,  13,   5,
         14,  13,   5,  14,  15,   6,  16,  21,   6,  20,  13,   5,  14,   7,
          8, 

In [146]:
x = og_tokenizer.decode(output_tokens[0])

x = x.split(" ")
for i in range(len(x)):
    if x[-1] == "[PAD]":
        x = x[:-1]
    else:
        break
x = " ".join(x)
x

'PIECE_START STYLE=JSFAKES GENRE=JSFAKES TRACK_START INST=0 BAR_START NOTE_ON=70 TIME_DELTA=4 NOTE_OFF=70 NOTE_ON=77 TIME_DELTA=4 NOTE_OFF=77 NOTE_ON=74 TIME_DELTA=4 NOTE_OFF=74 NOTE_ON=72 TIME_DELTA=2 NOTE_OFF=72 NOTE_ON=74 TIME_DELTA=2 NOTE_OFF=74 BAR_END BAR_START NOTE_ON=75 TIME_DELTA=4 NOTE_OFF=60 NOTE_ON=72 TIME_DELTA=8 NOTE_OFF=72 NOTE_ON=70 TIME_DELTA=4 NOTE_OFF=70 BAR_END BAR_START NOTE_ON=70 TIME_DELTA=4 NOTE_OFF=70 NOTE_ON=65 TIME_DELTA=4 NOTE_OFF=65 NOTE_ON=67 TIME_DELTA=4 NOTE_OFF=67 NOTE_ON=69 TIME_DELTA=4 NOTE_OFF=69 BAR_END BAR_START NOTE_ON=70 TIME_DELTA=2 NOTE_OFF=70 NOTE_ON=69 TIME_DELTA=2 NOTE_OFF=69 NOTE_ON=67 TIME_DELTA=8 NOTE_OFF=48 NOTE_ON=65 TIME_DELTA=4 NOTE_OFF=65 BAR_END TRACK_END TRACK_START INST=32 BAR_START NOTE_ON=58 TIME_DELTA=4 NOTE_OFF=58 NOTE_ON=57 TIME_DELTA=4 NOTE_OFF=57 NOTE_ON=58 TIME_DELTA=4 NOTE_OFF=58 NOTE_ON=53 TIME_DELTA=4 NOTE_OFF=53 BAR_END BAR_START NOTE_ON=51 TIME_DELTA=4 NOTE_OFF=51 NOTE_ON=53 TIME_DELTA=8 NOTE_OFF=53 NOTE_OFF=41 TIME_D

In [162]:
import mido

# Parse the musical notation data
music_string = x
events = music_string.split(' ')

# Create a MIDI file
midi_file = mido.MidiFile()

# Add MIDI events to the file
for event in events:
    # Process the event and add MIDI messages to the file
    pass  # You'll need to implement this part based on your specific notation format

# Save the MIDI file
# midi_file.save('output.mid')

In [150]:
import note_seq

NOTE_LENGTH_16TH_120BPM = 0.25 * 60 / 120
BAR_LENGTH_120BPM = 4.0 * 60 / 120

def token_sequence_to_note_sequence(token_sequence, use_program=True, use_drums=True, instrument_mapper=None, only_piano=False):

    if isinstance(token_sequence, str):
        token_sequence = token_sequence.split()

    note_sequence = empty_note_sequence()

    # Render all notes.
    current_program = 1
    current_is_drum = False
    current_instrument = 0
    track_count = 0
    for token_index, token in enumerate(token_sequence):

        if token == "PIECE_START":
            pass
        elif token == "PIECE_END":
            print("The end.")
            break
        elif token == "TRACK_START":
            current_bar_index = 0
            track_count += 1
            pass
        elif token == "TRACK_END":
            pass
        elif token == "KEYS_START":
            pass
        elif token == "KEYS_END":
            pass
        elif token.startswith("KEY="):
            pass
        elif token.startswith("INST"):
            instrument = token.split("=")[-1]
            if instrument != "DRUMS" and use_program:
                if instrument_mapper is not None:
                    if instrument in instrument_mapper:
                        instrument = instrument_mapper[instrument]
                current_program = int(instrument)
                current_instrument = track_count
                current_is_drum = False
            if instrument == "DRUMS" and use_drums:
                current_instrument = 0
                current_program = 0
                current_is_drum = True
        elif token == "BAR_START":
            current_time = current_bar_index * BAR_LENGTH_120BPM
            current_notes = {}
        elif token == "BAR_END":
            current_bar_index += 1
            pass
        elif token.startswith("NOTE_ON"):
            pitch = int(token.split("=")[-1])
            note = note_sequence.notes.add()
            note.start_time = current_time
            note.end_time = current_time + 4 * NOTE_LENGTH_16TH_120BPM
            note.pitch = pitch
            note.instrument = current_instrument
            note.program = current_program
            note.velocity = 80
            note.is_drum = current_is_drum
            current_notes[pitch] = note
        elif token.startswith("NOTE_OFF"):
            pitch = int(token.split("=")[-1])
            if pitch in current_notes:
                note = current_notes[pitch]
                note.end_time = current_time
        elif token.startswith("TIME_DELTA"):
            delta = float(token.split("=")[-1]) * NOTE_LENGTH_16TH_120BPM
            current_time += delta
        elif token.startswith("DENSITY="):
            pass
        elif token == "[PAD]":
            pass
        else:
            #print(f"Ignored token {token}.")
            pass

    # Make the instruments right.
    instruments_drums = []
    for note in note_sequence.notes:
        pair = [note.program, note.is_drum]
        if pair not in instruments_drums:
            instruments_drums += [pair]
        note.instrument = instruments_drums.index(pair)

    if only_piano:
        for note in note_sequence.notes:
            if not note.is_drum:
                note.instrument = 0
                note.program = 0

    return note_sequence

def empty_note_sequence(qpm=120.0, total_time=0.0):
    note_sequence = note_seq.protobuf.music_pb2.NoteSequence()
    note_sequence.tempos.add().qpm = qpm
    note_sequence.ticks_per_quarter = note_seq.constants.STANDARD_PPQ
    note_sequence.total_time = total_time
    return note_sequence

In [178]:
y = token_sequence_to_note_sequence(x)
note_seq.play_sequence(y)


c:\Users\joeac\AppData\Local\Programs\Python\Python311\Lib\site-packages\pretty_midi\pretty_midi.py:947: RuntimeWarning: invalid value encountered in divide
  synthesized /= np.abs(synthesized).max()
C:\Users\joeac\AppData\Roaming\Python\Python311\site-packages\IPython\lib\display.py:188: RuntimeWarning: invalid value encountered in cast
  return scaled.astype("<h").tobytes(), nchan


AttributeError: play